In [1]:
import ee
import os
import geemap

service_account = 'jupyter-gee-project@ee-avs-dse.iam.gserviceaccount.com'
key_path = '../service_account_key.json'

credentials = ee.ServiceAccountCredentials(service_account, key_path)
ee.Initialize(credentials)

print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


In [ ]:
protected_areas = ee.FeatureCollection("WCMC/WDPA/202106/polygons")
filtered = protected_areas.filter(ee.Filter.And(
    ee.Filter.eq("MARINE", "0"),
    ee.Filter.neq("DESIG_ENG", "Marine Protected Area"),
    ee.Filter.inList("STATUS", ["Designated", "Established", "Inscribed"]),
    ee.Filter.neq("DESIG_ENG", "UNESCO-MAB Biosphere Reserve"),
    ee.Filter.gte("GIS_AREA", 200), 
    ee.Filter.inList("WDPA_PID", [
        "555655917", "555656005", "555656013", "555665477", "555656021",
        "555665485", "555556142", "187", "555703455", "555563456", "15894"
    ]).Not()
))



In [ ]:
ogr2ogr -f "PostgreSQL" \
  PG:"host=host.docker.internal dbname=postgres user=postgres password=gispass" \
  protected_areas_filtered.geojson \
  -nln protected_areas \
  -nlt MULTIPOLYGON \
  -lco GEOMETRY_NAME=geom \
  -lco FID=gid \
  -overwrite

In [ ]:
-- Add a column for perimeter/area ratio
ALTER TABLE protected_areas ADD COLUMN pa_ratio DOUBLE PRECISION;

-- Compute area and perimeter using a projected CRS (Mollweide: ESRI:54009)
UPDATE protected_areas
SET pa_ratio = ST_Perimeter(ST_Transform(geom, 54009)) /
               ST_Area(ST_Transform(geom, 54009));

-- Remove top 25% by ratio
DELETE FROM protected_areas
WHERE pa_ratio > (
  SELECT percentile_cont(0.75) WITHIN GROUP (ORDER BY pa_ratio)
  FROM protected_areas
);